## [Mar 2] Computation Theory III

Presenter: Yuchen Ge  
Affiliation: University of Oxford  
Contact Email: gycdwwd@gmail.com  
Website: https://yuchenge-am.github.io

---

### 1. Graph Representation Learning

Assume edges are unweighted and undirectional. Each node  $v_{i} \in \mathcal{V}$ (each node is a token)  associates  $D$-dimensional feature vector  $\mathbf{x}_{i} \in \mathbb{R}^{1 \times D}$, and then stack the feature vectors into a single matrix $\mathbf{X} \in \mathbb{R}^{N \times D}$. A message passing GNN layer  $l$  is computed as

$$\mathbf{x}_{i}^{(l+1)}=\phi\left(\mathbf{x}_{i}^{(l)}, \bigoplus_{j \in \mathcal{N}\left(v_{i}\right)} \psi\left(\mathbf{x}_{i}^{(l)}, \mathbf{x}_{j}^{(l)}\right)\right)$$

where $\bigoplus$ is a  permutation-invariant aggregation function. In practice, we convolve node features with its neighbours 

$$\left(\mathbf{A X}^{(l)}\right)_{i}=\sum_{j \in \mathcal{N}\left(v_{i}\right)} \mathbf{x}_{j}^{(l)} \quad \text{ and } \quad \left(\hat{\mathbf{A}} \mathbf{X}^{(l)}\right)_{i}=\mathbf{x}_{i}^{(l)}+\sum_{j \in \mathcal{N}\left(v_{i}\right)} \mathbf{x}_{j}^{(l)}.
$$

> For GCNs, we use $\underbrace{\mathbf{X}^{(l+1)}}_{N \times D_{\text {out }}}=\sigma(\underbrace{\tilde{\mathbf{A}}}_{N \times N} \underbrace{\mathbf{X}^{(l)}}_{N \times D_{i n}} \underbrace{\mathbf{W}^{(l)}}_{D_{\text {in }} \times D_{\text {out }}})=\sigma(\underbrace{\hat{\mathbf{D}}^{-1 / 2}}_{N \times N} \underbrace{\hat{\mathbf{A}}}_{N \times N} \underbrace{\hat{\mathbf{D}}^{-1 / 2}}_{N \times N} \underbrace{\mathbf{X}^{(l)}}_{N \times D_{i n}} \underbrace{\mathbf{W}^{(l)}}_{D_{i n} \times D_{\text {out }}})$.

>
>

Concretely, $\left(\hat{\mathbf{D}}^{-1 / 2} \hat{\mathbf{A}} \hat{\mathbf{D}}^{-1 / 2} \mathbf{X}^{(l)} \mathbf{W}^{(l)}\right)_{i}=\left(\hat{\mathbf{D}}^{-1 / 2} \hat{\mathbf{A}} \hat{\mathbf{D}}^{-1 / 2} \mathbf{X}^{(l)}\right)_{i} \mathbf{W}^{(l)}=\left(c_{i i} \mathbf{x}_{i}^{(l)}+\sum_{j \in \mathcal{N}\left(v_{i}\right)} c_{i j} \mathbf{x}_{j}^{(l)}\right) \mathbf{W}^{(l)}$ where $c_{i j}=\frac{1}{\sqrt{\operatorname{deg}\left(v_{i}\right) \operatorname{deg}\left(v_{j}\right)}}$. And $\mathbf{W}^{(l)}$ simply performs a linear projection of the input features.


> For multihead GATs, we use $\underbrace{\mathbf{X}^{(l+1)}}_{N \times D_{\text {out }}}=\sigma(\frac{1}{K} \sum_{k=1}^{K} \underbrace{\tilde{\mathbf{A}}^{(l, k)}}_{N \times N} \underbrace{\mathbf{X}^{(l)}}_{N \times D_{\text {in }}} \underbrace{\mathbf{W}^{(l, k)}}_{D_{\text {in }} \times D_{\text {out }}}) $ where $\tilde{\mathbf{A}}^{(l, k)}=\tilde{\mathbf{A}}^{(l, k)}\left(\mathbf{X}^{(l)}\right)$.

Intuitively, the operations performed by the transformer block are

$$ \mathrm{h}=\operatorname{MHSA}(\operatorname{LayerNorm}(\mathrm{x}))+\mathrm{x}, \text{ and } \mathrm{y}=\operatorname{MLP}(\operatorname{LayerNorm}(\mathrm{h}))+\mathrm{h},$$

which are complemented with LayerNorms and skip connections aidding in training. 

> In words, MHSA processes the input feature matrix $\mathbf{X} \in \mathbb{R}^{N \times D}$ column-wise, whereas MLP row-wise.

Given  $\mathbf{Q}^{(l)}$,  $\mathbf{K}^{(l)}$, and  $\mathbf{V}^{(l)}$  are the query, key, and value matrices at layer  $l$, for MHSA, we introduce transformations below

$$\mathbf{Q}^{(l, k)}=\mathbf{X}^{(l)} \mathbf{W}_{Q}^{(l, k)} ; \quad \mathbf{K}^{(l, k)}=\mathbf{X}^{(l)} \mathbf{W}_{K}^{(l, k)} ; \quad \mathbf{V}^{(l, k)}=\mathbf{X}^{(l)} \mathbf{W}_{V}^{(l, k)}.$$


Then for each head $k$, $\tilde{\mathbf{A}}^{(l, k)}=\operatorname{softmax}\left(\frac{\mathbf{Q}^{(l, k)}\left(\mathbf{K}^{(l, k)}\right)^{T}}{\sqrt{D_{k}}}\right)$ descrbes the cross-token interaction. The output for each head is $\mathbf{H}^{(l, k)}=\tilde{\mathbf{A}}^{(l, k)} \mathbf{V}^{(l, k)}$. Next the output of all attention heads are concatenated and projected

$$\underbrace{\mathbf{X}^{(l+1)}}_{N \times D_{\text {out }}}=\underbrace{\left(\|_{k=1}^{K} \mathbf{H}^{(l, k)}\right)}_{N \times K D_{\text {out }}} \underbrace{\mathbf{W}_{O}^{(l)}}_{K D_{\text {out }} \times D_{\text {out }}}.$$


Here all $W_{\square}$  are learnable.

---


### 2. Skip connections, Layer Normalization, and Other Details

Skip Connections allow the information from earlier layers to be directly passed to later layers, mitigating the vanishing gradient problem.

Mathematically, write $z=l(x)+x$. Then 

$$\frac{\partial \mathcal{L}}{\partial W^{(l)}}=\frac{\partial \mathcal{L}}{\partial z^{(L)}} \cdot \ldots \cdot \frac{\partial z^{(l+1)}}{\partial z^{(l)}} \cdot \frac{\partial z^{(l)}}{\partial W^{(l)}}$$

where $\frac{\partial \mathcal{L}}{\partial W^{(l)}} =$ the parameters $W^{(l)}$ of layer $l$. Then we know 

$$ \frac{\partial z^{(l+1)}}{\partial z^{(l)}}=\frac{\partial\left(l\left(z^{(l)}\right)+z^{(l)}\right)}{\partial z^{(l)}}=\frac{\partial l\left(z^{(l)}\right)}{\partial z^{(l)}}+\frac{\partial z^{(l)}}{\partial z^{(l)}}=0+1 \rightarrow \frac{\partial \mathcal{L}}{\partial W^{(l)}} \neq 0. $$

LayerNorm enhances training stability and convergence speed, which operates by normalizing neuron activations within each layer independently. Specifically, 

$$\operatorname{LayerNorm}\left(x_{i}\right)=\gamma_{i} \frac{x_{i}-\mu_{i}}{\sqrt{\sigma_{i}^{2}+\epsilon}}+\beta_{i}$$

where $\epsilon > 0$ ensures numerical stability.

> BatchNorm calculates statistics across instances in a batch, while LayerNorm calculates statistics across the feature dimension.

---

### 3. Graph Transformers Model

In NLP, Transformers embrace full attention when constructing feature representations for words, treating sentences as fully connected graphs.

> Note: scale allows for computationally feasible attention w.r.t. memory/time requirements. (not necessarily hold true for Graph Transformers)

However, ggraph datasets exhibit arbitrary connectivity structures. Modeling of long-range dependencies and interactions between nodes is achieved by **global attention**.

Specifically, 

$$\begin{aligned}
\mathrm{h}= & \text { GlobalAttention }(\operatorname{LayerNorm}(\mathrm{x}))+\operatorname{MPNN}(\operatorname{LayerNorm}(\mathrm{x}, \mathrm{A}))+\mathrm{x} \text {, } \\
\mathrm{y}= & \operatorname{ MLP}(\operatorname{LayerNorm}(\mathrm{h}))+\mathrm{h} .
\end{aligned}$$

Graph Transformers address prevalent shortcomings of traditional message passing GNNs, including
over-smoothing, over-squashing, and limited expressivity. 

>  **Local message-passing and Expressivity**: the expressivity of GNNs is bounded by the WL test (test group isomorphism).
>
> **Over-smoothing**: multiple layers promotes interacting nodes going into higher-order neighborhoods and having similar feature, making interacting nodes become indistinguishable.
>
> **Over-squashing**：as receptive field of nodes exponentially expands, representations must be compressed into fixed-size vectors. Then over-squashing occurs when messages exchanged between distant nodes become distorted.

In traditional Transformers, positional encodings are added to the input embeddings.

> $$\operatorname{PE}(p, 2 d)=\sin \left(\frac{p}{10000^{(2 d / D)}}\right) \quad \text{ and } \quad \mathrm{PE}(p, 2 d+1)=\cos \left(\frac{p}{10000^{(2 d / D)}}\right)$$ 
> 
> where $p = $ position of a token in the sequence and $D =$ \# dimensions of the model input.

**Laplace Eigenfunction Encodings** are the counterpart in graph transformers.

>  They discern between various graph structures and sub-structures, akin to interpreting them as the frequencies of resonance within the graph.

Specifically, we use $\mathbf{L} \Phi=\lambda \Phi$, whose  eigenvectors are $\|_{m=1}^{M} \Phi_{i m} \in \mathbb{R}^{N \times M}$. With each entry $\Phi_{i} \in \mathbb{R}^{1 \times M}$ corresponding to $v_{i}$. These features can be used to compute for instance, relative encodings between nodes  $v_{i}$  and  $v_{j}$  =  $d\left(v_{i}, v_{j}\right)=d\left(\Phi_{i}, \Phi_{j}\right)$.

---

### Reference

1. Elucidating Graph Neural Networks, Transformers, and Graph Transformers.
2. Relating Graph Neural Networks to Structural Causal Models. 
3. Fine-Tuning Graph Neural Networks via Graph Topology induced Optimal Transport.
4. What Improves the Generalization of Graph Transformer? A Theoretical Dive into Self-attention and Positional Encoding.
5. Advective Diffusion Transformers for Topological Generalization in Graph Learning.
6. Deep Prompt Tuning for Graph Transformers.